### Get MassIVE

In [10]:
from filter_datasets import *
import json
import ppx
from pyteomics import mztab

from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa

In [2]:
dbs = getAllDts()
df = pd.DataFrame(dbs)

# Arabidopsis positive control
df.species[df.species.str.contains('3702')]

126                 Arabidopsis thaliana (NCBITaxon:3702)
267                 Arabidopsis thaliana (NCBITaxon:3702)
527                 Arabidopsis thaliana (NCBITaxon:3702)
741                 Arabidopsis thaliana (NCBITaxon:3702)
770                 Arabidopsis thaliana (NCBITaxon:3702)
789                 Arabidopsis thaliana (NCBITaxon:3702)
870                 Arabidopsis thaliana (NCBITaxon:3702)
1046                            Arabidopsis thaliana|3702
1077    Mus musculus (NCBITaxon:10090);Arabidopsis tha...
1101                            Arabidopsis thaliana|3702
1132                Arabidopsis thaliana (NCBITaxon:3702)
1319                Arabidopsis thaliana (NCBITaxon:3702)
1484                Arabidopsis thaliana (NCBITaxon:3702)
1567    Mus musculus (NCBITaxon:10090);Escherichia col...
1839                Arabidopsis thaliana (NCBITaxon:3702)
2029                Arabidopsis thaliana (NCBITaxon:3702)
2030                Arabidopsis thaliana (NCBITaxon:3702)
2110          

In [4]:
# result from filterMassIVE
df = pd.read_csv('massive_plantid_proteomics.tsv', sep='\t')
df.head()

,dataset,datasetNum,title,user,site,flowname,createdMillis,created,description,fileCount,...,pi,complete,quant_analysis,status,private,hash,px,task,id,search
0,MSV000087822,87822,TMK-based cell surface auxin signaling activat...,haiyanzheng,MassIVE,MASSIVE-COMPLETE,1626290200000,"Jul. 14, 2021, 12:16 PM",The phytohormone auxin controls a myriad of pr...,72,...,"[{'name': 'Yang, zhenbiao', 'email': 'zhenbiao...",False,NaN,Partial,False,NaN,NaN,ab62b7b1d5e5455dac665ed8629caf97,119,0
1,MSV000087472,87472,Complement of proteins associated with clathri...,dahhan,MassIVE,MASSIVE-COMPLETE,1621570970000,"May. 20, 2021, 9:22 PM","In Eukaryotes, clathrin coated vesicles (CCVs)...",43,...,"[{'name': 'Sebastian Y. Bednarek', 'email': 's...",True,NaN,Complete,False,NaN,PXD026180,d36fa49fcec846a288df2ffd04669068,260,1
2,MSV000086794,86794,Changes in mitochondrial proteome upon RNAi kn...,minsookim,MassIVE,MASSIVE-COMPLETE,1612266144000,"Feb. 2, 2021, 3:42 AM",To investigate how reduced ATP synthase subuni...,21,...,"[{'name': 'Elizabeth Vierling', 'email': 'vier...",False,NaN,Partial,False,NaN,NaN,b9b6448f3c534d37ae178b646f45ac3b,519,1
3,MSV000086324,86324,NOT9 Co-IP in Arabidopsis thaliana,klement,MassIVE,MASSIVE-COMPLETE,1603120368000,"Oct. 19, 2020, 8:12 AM",Interacting partners of NOT9 in Arabidopsis th...,13,...,"[{'name': 'Eva Klement', 'email': 'klement.eva...",False,NaN,Partial,False,NaN,NaN,910cf186e64d4d8593e9d5eecefc6c18,732,0
4,MSV000086248,86248,SA-Induced MTD-FLAG Interactome in A. thaliana,tch21,MassIVE,MASSIVE-COMPLETE,1602159199000,"Oct. 8, 2020, 5:13 AM",Identification of the salicylic acid (SA) indu...,506,...,"[{'name': 'John Williamson', 'email': 'jdw@ncs...",True,NaN,Complete,False,NaN,PXD021892,4b0caa7982ec4bbf9bc3c7c68ad1d6a2,760,0


### Get PRIDE

In [2]:
# result from filterPride

with open('pride_plant.json') as f:
    pride_plant = json.load(f)
    
len(pride_plant)

1464

In [3]:
identifiers = []
accession = []

for k,v in pride_plant.items():
    identifiers.append(k)
    accession.append(v['accession'])
    
identifiers[:5]

['PRD000037', 'PRD000044', 'PRD000051', 'PRD000084', 'PRD000096']

In [12]:
proj = ppx.find_project(identifiers[0])
remote_files = proj.remote_files()
print(remote_files)

['PRIDE_Exp_Complete_Ac_3197.xml.gz', 'PRIDE_Exp_Complete_Ac_3198.xml.gz', 'PRIDE_Exp_Complete_Ac_3199.xml.gz', 'PRIDE_Exp_Complete_Ac_3200.xml.gz', 'PRIDE_Exp_Complete_Ac_3201.xml.gz', 'PRIDE_Exp_Complete_Ac_3202.xml.gz', 'PRIDE_Exp_Complete_Ac_3203.xml.gz', 'PRIDE_Exp_Complete_Ac_3204.xml.gz', 'PRIDE_Exp_Complete_Ac_3205.xml.gz', 'PRIDE_Exp_Complete_Ac_3206.xml.gz', 'PRIDE_Exp_Complete_Ac_3207.xml.gz', 'PRIDE_Exp_Complete_Ac_3208.xml.gz', 'PRIDE_Exp_Complete_Ac_3209.xml.gz', 'PRIDE_Exp_Complete_Ac_3210.xml.gz', 'PRIDE_Exp_Complete_Ac_3211.xml.gz', 'PRIDE_Exp_Complete_Ac_3214.xml.gz', 'PRIDE_Exp_Complete_Ac_3215.xml.gz', 'PRIDE_Exp_Complete_Ac_3216.xml.gz', 'PRIDE_Exp_Complete_Ac_3217.xml.gz', 'README.txt']


In [17]:
findmztab(identifiers[0])==None

True

In [20]:
for i in identifiers:
    if findmztab(i)!=None:
        print(i)
        break

PRD000084


In [21]:
proj = ppx.find_project('PRD000084')

In [22]:
proj.metadata.keys()

dict_keys(['accession', 'title', 'additionalAttributes', 'projectDescription', 'sampleProcessingProtocol', 'dataProcessingProtocol', 'projectTags', 'keywords', 'doi', 'submissionType', 'submissionDate', 'publicationDate', 'submitters', 'labPIs', 'affiliations', 'instruments', 'softwares', 'quantificationMethods', 'countries', 'sampleAttributes', 'organisms', 'organismParts', 'diseases', 'references', 'identifiedPTMStrings', '_links'])

In [23]:
pride_plant['PRD000084']

{'name': 'Arabidopsis thaliana (mouse-ear cress)', 'accession': '3702'}

In [24]:
remote_files = proj.remote_files()
print(remote_files)

['PRIDE_Exp_Complete_Ac_15505.xml.gz', 'PRIDE_Exp_Complete_Ac_8743.xml.gz', 'PRIDE_Exp_Complete_Ac_8744.xml.gz', 'PRIDE_Exp_Complete_Ac_8745.xml.gz', 'PRIDE_Exp_Complete_Ac_8746.xml.gz', 'PRIDE_Exp_Complete_Ac_8747.xml.gz', 'PRIDE_Exp_Complete_Ac_8748.xml.gz', 'PRIDE_Exp_Complete_Ac_8749.xml.gz', 'PRIDE_Exp_Complete_Ac_8750.xml.gz', 'README.txt', 'generated/PRIDE_Exp_Complete_Ac_15505.pride.mgf.gz', 'generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz', 'generated/PRIDE_Exp_Complete_Ac_8743.pride.mgf.gz', 'generated/PRIDE_Exp_Complete_Ac_8743.pride.mztab.gz', 'generated/PRIDE_Exp_Complete_Ac_8744.pride.mgf.gz', 'generated/PRIDE_Exp_Complete_Ac_8744.pride.mztab.gz', 'generated/PRIDE_Exp_Complete_Ac_8745.pride.mgf.gz', 'generated/PRIDE_Exp_Complete_Ac_8745.pride.mztab.gz', 'generated/PRIDE_Exp_Complete_Ac_8746.pride.mgf.gz', 'generated/PRIDE_Exp_Complete_Ac_8746.pride.mztab.gz', 'generated/PRIDE_Exp_Complete_Ac_8747.pride.mgf.gz', 'generated/PRIDE_Exp_Complete_Ac_8747.pride.mztab.gz', '

In [25]:
findmztab(i)

{'PRD000084': ['generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8743.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8744.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8745.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8746.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8747.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8748.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8749.pride.mztab.gz',
  'generated/PRIDE_Exp_Complete_Ac_8750.pride.mztab.gz']}

In [26]:
downloaded = proj.download("generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz")
print(downloaded)

TOTAL:   0%|                                                                                                                                                                              | 0/1 [00:00<?, ?files/s]
generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz:   0%|                                                                                                                             | 0.00/480k [00:00<?, ?b/s]
generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz:   0%|▎                                                                                                                   | 1.41k/480k [00:01<07:45, 1.05kb/s]
generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz:   3%|███▊                                                                                                                | 15.6k/480k [00:01<00:36, 13.0kb/s]
generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz:   9%|██████████▌                                                                                  

[PosixPath('/home/rsilva/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz')]


In [27]:
!gzip -d /home/rsilva/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz

In [29]:
!ls /home/rsilva/.ppx/PRD000084/generated/

PRIDE_Exp_Complete_Ac_15505.pride.mztab


In [11]:
tables = mztab.MzTab("/home/rsilva/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab")
psms = tables.spectrum_match_table

psms.head()

,sequence,PSM_ID,accession,unique,database,database_version,search_engine,search_engine_score[1],search_engine_score[2],search_engine_score[3],...,retention_time,charge,exp_mass_to_charge,calc_mass_to_charge,spectra_ref,pre,post,start,end,opt_global_cv_MS:1002217_decoy_peptide
PSM_ID,,,,,,,,,,,,,,,,,,,,,
3964,DNPPPAGLPPNSGAGAGGAQSLIK,3964,AT5G07530,None,TAIR9,2009-06-15,"[PepSplice, SEQUEST, PeptideProphet]",6.1067,8.91990,0.0,...,None,2,1094.42,1093.564167,ms_run[1]:spectrum=3964,None,None,149.0,172.0,0
7112,DNPPPAGLPPNSGAGAGGAQSLIK,7112,AT5G07530,None,TAIR9,2009-06-15,"[SEQUEST, PeptideProphet]",3.0262,NaN,NaN,...,None,2,1093.90,1093.564167,ms_run[1]:spectrum=7112,None,None,149.0,172.0,0
1418,DNPPPAGLPPNSGAGAGGAQSLIK,1418,AT5G07530,None,TAIR9,2009-06-15,"[PepSplice, SEQUEST, PeptideProphet]",5.5238,8.17689,0.0,...,None,2,1095.45,1093.564167,ms_run[1]:spectrum=1418,None,None,149.0,172.0,0
9476,DNPPPAGLPPNSGAGAGGAQSLIK,9476,AT5G07530,None,TAIR9,2009-06-15,"[PepSplice, SEQUEST, PeptideProphet]",6.2424,14.40110,0.0,...,None,2,1094.61,1093.564167,ms_run[1]:spectrum=9476,None,None,149.0,172.0,0
4778,DNPPPAGLPPNSGAGAGGAQSLIK,4778,AT5G07530,None,TAIR9,2009-06-15,"[PepSplice, SEQUEST, PeptideProphet]",4.5757,18.49130,0.0,...,None,2,1093.68,1093.564167,ms_run[1]:spectrum=4778,None,None,149.0,172.0,0


In [12]:
psms.columns

Index(['sequence', 'PSM_ID', 'accession', 'unique', 'database',
       'database_version', 'search_engine', 'search_engine_score[1]',
       'search_engine_score[2]', 'search_engine_score[3]',
       'search_engine_score[4]', 'search_engine_score[5]',
       'search_engine_score[6]', 'search_engine_score[7]',
       'search_engine_score[8]', 'modifications', 'retention_time', 'charge',
       'exp_mass_to_charge', 'calc_mass_to_charge', 'spectra_ref', 'pre',
       'post', 'start', 'end', 'opt_global_cv_MS:1002217_decoy_peptide'],
      dtype='object')

In [13]:
tables.keys()

odict_keys(['PRT', 'PEP', 'PSM', 'SML'])

In [14]:
tables['PRT']['description'].str.contains('unknown').sum()

83

In [15]:
prt = pd.DataFrame(tables['PRT'])

In [16]:
prt.head()

,accession,description,taxid,species,database,database_version,search_engine,best_search_engine_score[1],search_engine_score[1]_ms_run[1],num_psms_ms_run[1],...,num_peptides_unique_ms_run[1],ambiguity_members,modifications,protein_coverage,opt_global_cv_PRIDE:0000303_Decoy_hit,opt_global_num_merge_proteins,opt_global_duplicated_proteins_search_engines,opt_global_duplicated_proteins_best_search_engines_score,opt_global_duplicated_proteins_search_engines_scores,opt_global_duplicated_proteins_had_quantification
accession,,,,,,,,,,,,,,,,,,,,,
AT00116,AT00116,None,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,9,...,None,None,None,0.0,0,1,None,None,None,0
AT1G01090,AT1G01090,PDH-E1 ALPHA (PYRUVATE DEHYDROGENASE E1 ALPHA)...,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,2,...,None,None,101-MOD:00397,0.0,0,1,None,None,None,0
AT1G01140,AT1G01140,CIPK9 (CBL-INTERACTING PROTEIN KINASE 9); ATP ...,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,1,...,None,None,None,0.0,0,1,None,None,None,0
AT1G02010,AT1G02010,SEC1A (SIMILAR TO ELECTRON CAREER 1A); protein...,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,4,...,None,None,"233-MOD:00397,249-MOD:00397",0.0,0,1,None,None,None,0
AT1G02080,AT1G02080,transcriptional regulator-related,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,1,...,None,None,None,0.0,0,1,None,None,None,0


In [17]:
prt['description'].str.contains('unknown').sum()

83

In [18]:
idx = prt['description'].str.contains('unknown')
idx

accession
AT00116       None
AT1G01090    False
AT1G01140    False
AT1G02010    False
AT1G02080    False
             ...  
ATMG00070    False
ATMG00160    False
ATMG00220    False
ATMG00510    False
ATMG00640    False
Name: description, Length: 1251, dtype: object

In [19]:
idx[0] = False
prt[idx]

,accession,description,taxid,species,database,database_version,search_engine,best_search_engine_score[1],search_engine_score[1]_ms_run[1],num_psms_ms_run[1],...,num_peptides_unique_ms_run[1],ambiguity_members,modifications,protein_coverage,opt_global_cv_PRIDE:0000303_Decoy_hit,opt_global_num_merge_proteins,opt_global_duplicated_proteins_search_engines,opt_global_duplicated_proteins_best_search_engines_score,opt_global_duplicated_proteins_search_engines_scores,opt_global_duplicated_proteins_had_quantification
accession,,,,,,,,,,,,,,,,,,,,,
AT1G02813,AT1G02813,unknown protein,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,2,...,None,None,None,0.0,0,1,None,None,None,0
AT1G04910,AT1G04910,unknown protein,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,1,...,None,None,None,0.0,0,1,None,None,None,0
AT1G07220,AT1G07220,FUNCTIONS IN: molecular_function unknown; INVO...,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,1,...,None,None,None,0.0,0,1,None,None,None,0
AT1G12150,AT1G12150,FUNCTIONS IN: molecular_function unknown; INVO...,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,1,...,None,None,None,0.0,0,1,None,None,None,0
AT1G19140,AT1G19140,FUNCTIONS IN: molecular_function unknown; INVO...,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,2,...,None,None,None,0.0,0,1,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AT5G53820,AT5G53820,unknown protein,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,1,...,None,None,None,0.0,0,1,None,None,None,0
AT5G55060,AT5G55060,unknown protein,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,1,...,None,None,None,0.0,0,1,None,None,None,0
AT5G61720,AT5G61720,unknown protein,None,None,TAIR9,2009-06-15,analysis software,0.0,0.0,86,...,None,None,"201-MOD:00412,60-MOD:00412,61-MOD:00397",0.0,0,1,None,None,None,0


In [20]:
prt['description'].str.contains('lnc').sum()

0

In [23]:
prt['description'].str.contains('coding', case=False).sum()

0

In [24]:
prt['description'].str.contains('non', case=False).sum()

17

In [6]:
accession = list(set(accession))

print(len(accession))
accession[:10]

271


['29722',
 '488668',
 '36774',
 '656367',
 '97441',
 '3668',
 '148545',
 '748693',
 '3702',
 '2066563']

In [9]:
ncbi = NCBITaxa()


tree = ncbi.get_topology(accession[:3])

print (tree.get_ascii(attributes=["sci_name", "rank"]))

In [ ]:
# Not working, make plot of species available, can use other tools
# https://phylot.biobyte.de
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180

for n in tree.traverse():
    if n.is_leaf():
        n.name = n.sci_name

tree.render("pride_plant_tree.pdf", w=500, tree_style=ts)